# Going From Staregies A&B To A Portfolio

So far, I have implemented two startegies using momentum and news analytics on Quantopian along with their backtests. The universe of assets is considered to be Q1500US - the 1500 most liquid equities in the US stock exchange. In this notebook I will attempt to take the best performing equities from those startegies for my portfolio and size them appropriately

## Background Work

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
from IPython.display import display

pd.options.display.max_rows = 5

### Defining All Equities

In [2]:
equities_strategyA = [
    'ARNC',
    'ABT',
    'ADSK',
    'ADBE',
    'ADI',
    'AEP',
    'AES',
    'AFL',
    'AMD',
    'ADP'
]

equities_strategyB = [
    'ENDP',
    'MDCO',
    'WW',
    'STMP',
    'INFN',
    'SEMG',
    'MIK',
    'RETA',
    'CLDR',
    'HOME',
]

### Querying Data From Yahoo Finance

In [3]:
end = datetime.now().strftime("%Y-%m-%d")
start = datetime.now() - relativedelta(years=1)


data_strategyA = yf.download(equities_strategyA,
                             start,
                             end)['Close']
data_strategyB = yf.download(equities_strategyB,
                             start,
                             end)['Close']

print("\nFields:", data_strategyA.columns.get_level_values(0).unique())

display(data_strategyA)
display(data_strategyB)

data_strategyA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
data_strategyB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed

Fields: Index(['ABT', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AES', 'AFL', 'AMD',
       'ARNC'],
      dtype='object')


,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2018-11-30,74.050003,250.889999,91.919998,147.419998,144.500000,77.739998,15.490000,45.740002,21.299999,21.480000
2018-12-03,74.269997,255.259995,93.510002,147.410004,148.020004,78.309998,15.600000,46.070000,23.709999,21.520000
...,...,...,...,...,...,...,...,...,...,...
2019-11-27,85.419998,309.059998,113.699997,171.479996,180.179993,91.589996,18.889999,54.820000,39.410000,31.260000
2019-11-29,85.449997,309.529999,112.949997,170.779999,180.899994,91.349998,18.910000,54.840000,39.150002,30.959999


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2018-11-30,12.34,12.03,28.50,4.31,22.129999,16.969999,63.150002,16.230000,171.460007,50.020000
2018-12-03,12.31,12.24,29.15,4.38,21.820000,17.430000,67.080002,16.950001,170.100006,51.849998
...,...,...,...,...,...,...,...,...,...,...
2019-11-27,9.95,4.78,8.57,6.47,84.050003,8.200000,195.710007,15.400000,88.900002,44.290001
2019-11-29,9.88,5.08,8.58,6.38,84.199997,8.180000,195.130005,15.370000,87.250000,43.290001


### Volatility Calculation

In [4]:
volatilityA = data_strategyA.rolling(22).std().dropna(how='all')
volatilityB = data_strategyB.rolling(22).std().dropna(how='all')

display(volatilityA)
display(volatilityB)

volatilityA.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyA')
volatilityB.plot(figsize=(15,5)); plt.grid(); plt.title('StrategyB')

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
Date,,,,,,,,,,
2019-01-03,2.226695,14.538458,3.402288,7.265215,7.580474,2.398800,0.683810,1.096895,1.733212,1.676804
2019-01-04,2.121670,13.922237,3.286367,6.685046,7.000931,2.512728,0.678449,1.056491,1.669451,1.574362
...,...,...,...,...,...,...,...,...,...,...
2019-11-27,0.898435,10.404885,2.301155,4.542952,8.893375,1.708667,0.621717,0.471482,2.400075,1.231198
2019-11-29,0.950051,9.942688,2.178806,4.561418,9.621265,1.648445,0.611560,0.498858,2.219657,1.169370


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
Date,,,,,,,,,,
2019-01-03,0.824446,1.891282,3.656077,0.259816,1.756616,1.377411,4.768085,1.059908,9.981389,4.551310
2019-01-04,0.817239,1.825581,3.170837,0.257169,1.675728,1.275324,4.590542,1.036778,9.589692,4.667130
...,...,...,...,...,...,...,...,...,...,...
2019-11-27,0.420842,0.245794,0.317564,0.554211,11.329288,0.587983,8.038263,0.503917,2.348886,3.638282
2019-11-29,0.462606,0.272584,0.283806,0.543688,12.333963,0.563785,8.263487,0.503678,2.335218,3.834812


### Getting Asset Correlation (from Close Price)

In [5]:
corrA = data_strategyA.corr()
corrB = data_strategyB.corr()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(corrA)
    display(corrB)

,ABT,ADBE,ADI,ADP,ADSK,AEP,AES,AFL,AMD,ARNC
ABT,1.000000,0.902729,0.842327,0.889276,0.448717,0.875491,0.323860,0.847734,0.865540,0.785079
ADBE,0.902729,1.000000,0.850561,0.912815,0.695994,0.790401,0.421111,0.877036,0.881186,0.747757
ADI,0.842327,0.850561,1.000000,0.851439,0.684538,0.732105,0.538615,0.748894,0.756890,0.584690
ADP,0.889276,0.912815,0.851439,1.000000,0.659767,0.855423,0.435280,0.828005,0.876482,0.713842
ADSK,0.448717,0.695994,0.684538,0.659767,1.000000,0.338399,0.669079,0.606531,0.516125,0.222202
AEP,0.875491,0.790401,0.732105,0.855423,0.338399,1.000000,0.207403,0.823568,0.848193,0.854042
AES,0.323860,0.421111,0.538615,0.435280,0.669079,0.207403,1.000000,0.415726,0.428656,0.147804
AFL,0.847734,0.877036,0.748894,0.828005,0.606531,0.823568,0.415726,1.000000,0.858930,0.750408
AMD,0.865540,0.881186,0.756890,0.876482,0.516125,0.848193,0.428656,0.858930,1.000000,0.895509
ARNC,0.785079,0.747757,0.584690,0.713842,0.222202,0.854042,0.147804,0.750408,0.895509,1.000000


,CLDR,ENDP,HOME,INFN,MDCO,MIK,RETA,SEMG,STMP,WW
CLDR,1.000000,0.876478,0.883806,0.297996,-0.459611,0.834864,-0.136140,0.628376,0.717753,0.173582
ENDP,0.876478,1.000000,0.913768,-0.037483,-0.668351,0.925116,-0.291208,0.578510,0.751443,0.103131
HOME,0.883806,0.913768,1.000000,-0.069282,-0.644783,0.856594,-0.331761,0.496799,0.564715,-0.092215
INFN,0.297996,-0.037483,-0.069282,1.000000,0.524635,-0.011851,0.548953,0.357510,0.238487,0.509263
MDCO,-0.459611,-0.668351,-0.644783,0.524635,1.000000,-0.620319,0.768242,-0.044871,-0.453461,0.230279
MIK,0.834864,0.925116,0.856594,-0.011851,-0.620319,1.000000,-0.283189,0.684091,0.770081,0.224122
RETA,-0.136140,-0.291208,-0.331761,0.548953,0.768242,-0.283189,1.000000,0.248703,-0.172452,0.204890
SEMG,0.628376,0.578510,0.496799,0.357510,-0.044871,0.684091,0.248703,1.000000,0.586648,0.387747
STMP,0.717753,0.751443,0.564715,0.238487,-0.453461,0.770081,-0.172452,0.586648,1.000000,0.547992
WW,0.173582,0.103131,-0.092215,0.509263,0.230279,0.224122,0.204890,0.387747,0.547992,1.000000


## Using Calculations Above For Position Sizing

### Startegy A

In [6]:
volatilityMeanAdjusted = volatilityA.mean()/data_strategyA.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrA.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsA = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsA = position_weightsA/position_weightsA.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsA*100)

ABT     10.722470
ADBE     9.035760
ADI      7.923753
ADP     10.545706
ADSK     9.141126
AEP     13.446104
AES     13.571714
AFL     11.578001
AMD      5.616541
ARNC     8.418825
dtype: float64


### Streategy B

In [7]:
volatilityMeanAdjusted = volatilityB.mean()/data_strategyB.mean()
volatilityMeanAdjusted = (volatilityMeanAdjusted)/volatilityMeanAdjusted.sum()


corrAdjusted = corrB.sum(axis=1) - 1
corrAdjusted = (corrAdjusted-corrAdjusted.min())/corrAdjusted.sum()

position_weightsB = 1/(volatilityMeanAdjusted+corrAdjusted)
position_weightsB = position_weightsB/position_weightsB.sum()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(position_weightsB*100)

CLDR     7.010641
ENDP     6.855164
HOME     7.473356
INFN     9.020799
MDCO    29.652848
MIK      7.262836
RETA    10.924017
SEMG     7.064518
STMP     6.185964
WW       8.549858
dtype: float64

## Allocating Risk Based On Weights Calulated

So far we have weights (that sum up to 1) for each strategy. How do we decide how much capital to put into each strategy? Here I will allocate 1/2 of the risk into each strategy and size the entire portfolio accordingly. I will be working with the assumption that the max drawdown for both the strategies combined is 10M. Based on a 50% risk allocation, this will mean I need to size each strategy's positions such the drawdown for the strategy does not exceed 5M.

### Strategy A

In [8]:
df_A = pd.concat([data_strategyA.iloc[-1, :], position_weightsA, volatilityA.mean()],
          axis=1)
df_A.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_A['-2std'] = -2 * df_A['Avg Volatility']
df_A['Risk'] = 5e5 * df_A['Weight']
df_A['Shares'] = -(df_A['Risk']/df_A['-2std']).round()
df_A['Value Today'] = df_A['Shares'] * df_A['Price Today']

print('Total Dollars Invested Today: ', df_A['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_A)

Total Dollars Invested Today:  12230993.263866425


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
ABT,85.449997,0.107225,1.573037,-3.146073,53612.350696,17041.0,1.456153e+06
ADBE,309.529999,0.090358,6.774937,-13.549873,45178.801304,3334.0,1.031973e+06
ADI,112.949997,0.079238,3.479324,-6.958648,39618.764093,5693.0,6.430243e+05
ADP,170.779999,0.105457,3.036198,-6.072397,52728.529922,8683.0,1.482883e+06
ADSK,180.899994,0.091411,5.325145,-10.650290,45705.629476,4291.0,7.762419e+05
AEP,91.349998,0.134461,1.288470,-2.576940,67230.520798,26089.0,2.383230e+06
AES,18.910000,0.135717,0.446683,-0.893367,67858.569403,75958.0,1.436366e+06
AFL,54.840000,0.115780,0.884378,-1.768756,57890.003941,32729.0,1.794858e+06
AMD,39.150002,0.056165,1.407467,-2.814934,28082.703063,9976.0,3.905604e+05
ARNC,30.959999,0.084188,0.779722,-1.559443,42094.127305,26993.0,8.357033e+05


### Strategy B

In [9]:
df_B = pd.concat([data_strategyB.iloc[-1, :], position_weightsB, volatilityB.mean()],
          axis=1)
df_B.columns = ['Price Today', 'Weight', 'Avg Volatility']
df_B['-2std'] = -2 * df_B['Avg Volatility']
df_B['Risk'] = 5e5 * df_B['Weight']
df_B['Shares'] = -(df_B['Risk']/df_B['-2std']).round()
df_B['Value Today'] = df_B['Shares'] * df_B['Price Today']

print('Total Dollars Invested Today: ', df_B['Value Today'].sum())


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_B)

Total Dollars Invested Today:  5920494.353401184


,Price Today,Weight,Avg Volatility,-2std,Risk,Shares,Value Today
CLDR,9.880000,0.070106,0.650409,-1.300818,35053.204360,26947.0,2.662364e+05
ENDP,5.080000,0.068552,0.583211,-1.166422,34275.822390,29385.0,1.492758e+05
HOME,8.580000,0.074734,1.367975,-2.735950,37366.780801,13658.0,1.171856e+05
INFN,6.380000,0.090208,0.278807,-0.557615,45103.994315,80887.0,5.160591e+05
MDCO,84.199997,0.296528,1.964518,-3.929036,148264.237616,37736.0,3.177371e+06
MIK,8.180000,0.072628,0.778153,-1.556306,36314.179372,23334.0,1.908721e+05
RETA,195.130005,0.109240,8.571625,-17.143251,54620.082815,3186.0,6.216842e+05
SEMG,15.370000,0.070645,0.874653,-1.749306,35322.589002,20192.0,3.103510e+05
STMP,87.250000,0.061860,9.680743,-19.361486,30929.820104,1597.0,1.393382e+05
WW,43.290001,0.085499,2.141424,-4.282847,42749.289225,9982.0,4.321208e+05


## Portfolio (Optimized) 

In [10]:
portfolio = pd.concat([df_A[['Price Today', 'Shares', 'Value Today']], 
         df_B[['Price Today', 'Shares', 'Value Today']]],
         axis=0)
portfolio.columns = ['Price Today', 'Optimal Shares', 'Value Today']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(portfolio)

,Price Today,Optimal Shares,Value Today
ABT,85.449997,17041.0,1.456153e+06
ADBE,309.529999,3334.0,1.031973e+06
ADI,112.949997,5693.0,6.430243e+05
ADP,170.779999,8683.0,1.482883e+06
ADSK,180.899994,4291.0,7.762419e+05
AEP,91.349998,26089.0,2.383230e+06
AES,18.910000,75958.0,1.436366e+06
AFL,54.840000,32729.0,1.794858e+06
AMD,39.150002,9976.0,3.905604e+05
ARNC,30.959999,26993.0,8.357033e+05


In [11]:
portfolio['Value Today'].sum()/1e6

18.15148761726761

## What About the Current Holdings?

At this point we have determined optimal position sizes based on the risk allocation from weights above. However, at the time, I have already submitted multiple trades. In this section I will investigate wheather the worst case, for the current risk allocation, takes me less than 1M.

In [12]:
current_B = pd.DataFrame([95480,
8700,
12880,
5400,
79540,
27880,
52900,
2180,
52840,
51780],
                        index = equities_strategyB,
                        columns = ['Shares Holding'])

current_A = pd.DataFrame([15860,
6160,
2520,
2720,
3620,
3620,
21800,
6760,
14840,
2600], 
                        index = equities_strategyA,
                        columns = ['Shares Holding'])

current_A['Risk'] = df_A['-2std']*current_A['Shares Holding']
current_B['Risk'] = df_B['-2std']*current_B['Shares Holding']

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(current_A)
    display(current_B)
    
print('Worst Case Drawdown:', current_A['Risk'].sum() + current_B['Risk'].sum())

,Shares Holding,Risk
ARNC,15860,-24732.770328
ABT,6160,-19379.812305
ADSK,2520,-26838.730864
ADBE,2720,-36855.654992
ADI,3620,-25190.306866
AEP,3620,-9328.523280
AES,21800,-19475.400539
AFL,6760,-11956.790105
AMD,14840,-41773.614049
ADP,2600,-15788.231279


,Shares Holding,Risk
ENDP,95480,-111369.991630
MDCO,8700,-34182.616622
WW,12880,-55163.075349
STMP,5400,-104552.024184
INFN,79540,-44352.666786
SEMG,27880,-48770.650532
MIK,52900,-82328.613715
RETA,2180,-37372.286662
CLDR,52840,-68735.205892
HOME,51780,-141667.505381


Worst Case Drawdown: -959814.4713607125


I'm barely meeting the criteria - I have my current (suboptimally weighted) portfolio structure such that the worst case drawdown is less than -1M. So for now I will keep my positions.